In [58]:
import json
import torch
import evaluate
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [59]:
model_path = "/data/disk1/guohaoran/model/self/Qwen2.5-3B-Instruct"
device = 'cuda:7'

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float64,
    device_map=device,
)
print(model)
print(tokenizer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-35): 36 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear(in_features=2048, out_features=256, bias=True)
          (v_proj): Linear(in_features=2048, out_features=256, bias=True)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (up_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
      )
    )
    (norm):

In [60]:
tag_map = {
    'human': 'user',
    'gpt': 'assistant',
    'system': 'system',
    'function_call': 'function',
    'observation': 'observation',
}

def process(row):
    data = json.loads(row['text'])
    messages = [{'role': tag_map[line['from']], 'content': line['value']} for line in data['conversations']]
    text = tokenizer.apply_chat_template(messages, tokenize=False)
    result = tokenizer([text], return_tensors="np", padding='max_length', max_length=1024, truncation=True)
    result.update({'id': data['id'], 'text': text, 'length': len(text)})
    return result

dataset = load_dataset(
    path='text',
    data_files=['/data/disk1/guohaoran/data/sharegpt_zh_38K.jsonl'],
    cache_dir='/data/disk1/guohaoran/data/.cache',
    split='train',
)
dataset = dataset.map(process)
dataset = dataset.train_test_split(test_size=0.1)

train_dataset, test_dataset = dataset['train'], dataset['test']
train_dataset = train_dataset.shuffle()

print(dataset)
item = train_dataset[0]
print(item)


DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'id', 'length'],
        num_rows: 34701
    })
    test: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'id', 'length'],
        num_rows: 3856
    })
})
{'text': '<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\nhtml로 나의 첫번째 웹페어지를 만들어줘<|im_end|>\n<|im_start|>assistant\n当然！请按照以下方式编写代码。\n```php\n<!DOCTYPE html>\n<html>\n<head>\n\t<title>我的第一个网页</title>\n</head>\n<body>\n\t<h1>您好！</h1>\n\t<p>这是我的第一个网页。</p>\n\t<img src="https://via.placeholder.com/150" alt="替代文本">\n</body>\n</html>\n```\n此代码通过使用`<!DOCTYPE html>`声明HTML5文档，使用`<html>`标签标记HTML文档的开始，使用`<head>`标签声明文档标题的`<title>`标签。在`<body>`标签中，网页的内容被放置。`<h1>`标签表示标题，`<p>`标签表示正文。最后，`<img>`标签插入图片。图像通过`src`属性插入图像URL，通过`alt`属性放入备用文本，以避免当图像无法加载时使用默认备用文本。<|im_end|>\n<|im_start|>user\n请解释一下代码。<|im_end|>\n<|im_start|>assistant\n我会简单地介绍一下代码。\n```php\n<!DOCTYPE html>\n```\n*

In [61]:
training_args = Seq2SeqTrainingArguments(
    output_dir='/data/disk1/guohaoran/RhineAC/workspace/output',
    eval_strategy="epoch",
    per_device_train_batch_size=2,
    per_gpu_train_batch_size=2,
)
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


RuntimeError: expand(torch.cuda.DoubleTensor{[2, 2, 1, 1, 1024]}, size=[2, 1, 1, 1024]): the number of sizes provided (4) must be greater or equal to the number of dimensions in the tensor (5)